In [ ]:
import jupy_helpers

# Numpy = (fast) numerical Python

https://www.numpy.org says:
> NumPy is the fundamental package for scientific computing with Python. It contains among other things:
> 
> * a powerful N-dimensional array object
> * sophisticated (broadcasting) functions
> * tools for integrating C/C++ and Fortran code
> * useful linear algebra, Fourier transform, and random number capabilities

## Why Numpy?

Because numerics using only Python standard library is cumbersome and in most cases slow.

In [ ]:
import numpy as np

In [ ]:
np.__version__

The most used type is `np.ndarray`, which is often created by `np.array`.

In [ ]:
array = np.array([0, 2, 5])
array

In [ ]:
type(array)

Unlike built-in containers like `list`s, `np.ndarray`s have
* homogeneous `dtype`(s) (data types)

In [ ]:
array.dtype

* fixed `shape` (dimensions)

In [ ]:
array.shape

**Exercise:** Can you fill the missing values (by replacing `...`) to make the `assert`s pass (i.e. no `AssertException` raised)?

In [ ]:
test_array = np.array([[1.9, 2.8], [3.7, 4.5]])
test_array

In [ ]:
%exercise

# test_array_shape = ...
# test_array_dtype = ...

test_array_shape = (2, 2)
test_array_dtype = float

In [ ]:
%validate

assert test_array.shape == test_array_shape
assert test_array.dtype == test_array_dtype

Arrays can be created using helper constructors, such as `empty`, `zeros`, `arange`, `linspace`, `diag`, ...

In [ ]:
np.zeros((2, 8), dtype=float)

In [ ]:
np.linspace(-1, 1, 5)

**Exercise:** Explore what values are in a new `empty` array and try to explain.

In [ ]:
%exercise

# np.empty(...)

np.empty(10)

### Supported dtypes

- boolean 
- integers (8-, 16-, 32-, 64-bit), unsigned / signed
- float (32-, 64-bit)
- complex
- fixed-length strings (byte / unicode)
- object (~ pointers to any Python object)
- structured types (with more fields)

For more details, see https://docs.scipy.org/doc/numpy/user/basics.types.html and https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.htm

In [ ]:
string_array = np.array(["a", "aardvark", "abe"])
string_array.dtype

In [ ]:
string_array = np.array(["a", "aardvark", "abe"], dtype="object") 
string_array.dtype

## Buffer protocol

`ndarray` data are stored in (typically) contiguous memory blocks. This is for performance reasons because Numpy can employ fast C libraries (like BLAS) for vectorized operations without converting and / or rearranging the data in the memory. Python offers built-in [Buffer Protocol](https://docs.python.org/3/c-api/buffer.html#bufferobjects) which standardizes this approach and enables (relatively) easy integration with C libraries.

In a nutshell, `ndarray` data are described by their memory buffer location (pointer), and some metedata mainly describing the data type (`typestr`) and dimensionality (`shape` and `strides`).

Looking a bit under the hood, we can see the buffer info by `info` or `flags`

In [ ]:
np.info(array)

In [ ]:
array.flags

This is an implementation detail, although very important because it determines fundamental advantages, limitations and best practices for using `numpy`.

## Indexing

Numpy supports enhanced indexing compared e.g. to Python's `list` or `array` types. Especially:
* multiple dimensions
* enhaced slices
* "advanced" or "fancy" indexing using arrays

In [ ]:
array_1d = np.arange(0, 10)
array_1d

### Basic indexing 

Basic indexing and slicing on 1D array works identically to lists:

In [ ]:
array_1d[5]

In [ ]:
array_1d[5: -1]

In [ ]:
array_1d[::-3]

**Exercise:** Fill the slicing to satisfy the assert test. Notice we have to use `np.all` because the result is an array which cannot be converted to a single boolean value unambigously (is a true and false couple true or false?).

In [ ]:
%exercise

# result = array_1d[...]

result = array_1d[1::4]

In [ ]:
%validate

assert np.all(result == np.array([1, 5, 9]))

*Extra question:* Did you notice that (and how) the ellipsis slice `[...]` actually work?

Multi-dimensional arrays can be index conveniently:

In [ ]:
array_2d = np.random.randint(0, 10, (2, 6))
array_2d

In [ ]:
array_3d = array_2d.reshape(2, -1, 3)
array_3d

Single index returns a slice in one dimension:

In [ ]:
array_2d[1]

Which is equivalent to:

In [ ]:
array_2d[1, :]

And also

In [ ]:
array_2d[1, ...]

Examples for 3-D array:

In [ ]:
array_3d[:, 0, 1:]

In [ ]:
array_3d[..., -1]

### Fancy indexing

Suppose you need to access particular elements, say `3, 0, 5`. Instead of something like `np.array([array_1d[3], array_1d[0], array_1d[5]])`, you can write

In [ ]:
array_1d[[3, 0, 5]]

**Exercise:** Use (fancy) indexing to create a 8 x 8 chess-board pattern (0 / 1 in place of white / black) from a uniform array.

In [ ]:
%exercise

chess_board = np.ones((8, 8), dtype=int)

# chess_board[..., ...] = ...
# chess_board[..., ...] = ...

chess_board[::2, 1::2] = 0
chess_board[1::2, ::2] = 0

chess_board

In [ ]:
%validate

assert np.all(chess_board == np.tile([[1, 0], [0, 1]], (4, 4)))

Another possibility is to use arrays of boolean values for slicing.

In [ ]:
bool_indexes = np.zeros_like(array_1d, dtype=bool)
bool_indexes[[2, 5]] = True
bool_indexes

In [ ]:
array_1d[bool_indexes]

### Indexing vs. copying

Indexing in `numpy` does not have to create copies. Data copy is created only when inevitable.

Hence, **modyfying a slice can modify the original array**.

In [ ]:
original_array = np.linspace(0, 1, 3)
original_array

In [ ]:
array_slice = original_array[1: ]
array_slice *= 2
array_slice

In [ ]:
original_array

See that??? The original array has been modified.

## Vector operations

Numpy is optimized for performance. One of the key ingredient are vectorized operations: Instead of (slow) looping by elements, an operation can be performed on the whole or at least a subset of an array.

Let's try to implement a dot product (1D for simplicity):

In [ ]:
def dot(x, y):
    result = 0
    for i, j in zip(x, y):
        result += i * j
    
    return result

In [ ]:
x = np.random.sample((4, ))
y = np.random.sample((4, ))

dot(x, y)

Test our function using `allclose` to compare floats only up to certain precision.

In [ ]:
assert np.allclose(dot(x, y), x.dot(y))

We can compare the performance (on larger arrays)

In [ ]:
size = (1000, )
x_large = np.random.sample(size)
y_large = np.random.sample(size)

In [ ]:
%timeit dot(x_large, y_large)

In [ ]:
%timeit x_large.dot(y_large)

Even multiplacation (`*` is element-wise is Numpy) is vectorized. E.g.:

In [ ]:
x * y

**Exercise:** Implement dot using `*` and `sum` and measure its performace using `%timeit`.

In [ ]:
%exercise

# def dot_2(x, y):
    # return(___).___()

def dot_2(x, y):
    return (x * y).sum()

In [ ]:
%validate

assert np.allclose(dot_2(x, y), x.dot(y))

In [ ]:
%timeit dot_2(x_large, y_large)

Bottom line: use numpy functions and operations because they are vectorized and hence (in most cases) faster.

## Broadcasting

Numpy automatically broadcasts (aligns / enhances the dimensions) of arrays if it "makes sense" (there are strict rules :). Broadcasting simplifies operations and can make them faster and / or less memory consuming.

An example is to substract the column mean from an $n \times m$ array.

In [ ]:
X = np.linspace(0, 1, 15).reshape(-1, 3)
X

In [ ]:
X.mean(axis=0)

Now we'd like to center the columns around zero, i.e. make the mean = 0 of all columns.

In [ ]:
X_centered = X - X.mean(axis=0)

In [ ]:
X_centered.mean(axis=0)

We can also do this explicitely with `newaxis`

In [ ]:
X_centered = X - X.mean(axis=0)[np.newaxis, :]

Pandas uses broadcasting internally for many operations. That's for example why you should reserve some space in you RAM (around 3 - 5 times the dataset size), for a flawless Pandas data processing experience.

**Exercise:** Calculate $uv$ for random 1D arrays $u$ and $v$, defined as $(uv)_{ij} = u_i v_j$, using broadcasting.

In [ ]:
%exercise

u = np.random.sample((3, ))
v = np.random.sample((5, ))

# uv = u[...] * v[...]

uv = u[np.newaxis,:] * v[:,np.newaxis]

In [ ]:
%validate

for i, j in zip(range(u.size), range(v.size)):
    assert uv[i, j] == u[i] * v[j]